# Clustering JGI Publications - Template

Notebook intended to provide a framework for a clustering workflow to be used with titles and abstracts of JGI-enabled publications.

The __major goal__ if this project is to see if we can thematically group a set of publications. Ideally, our groupings will be as close to reality as possible. If a subject matter expert (i.e. a plant biologist) were to look at your model's results, would that person agree with them or at least judge them to be coherent topics that *could* emerge from a group of papers in their domain?

Text analysis is useful for impact-assessment because it allows us to more efficiently summarize / categorize the scientific activities of JGI users. If we spend a lot of money on a given project, it makes sense to ask what on earth people are actually using the resulting products for. Are there counter-intuitive uses that we didn't originally think of? Do some projects lend themselves better to use cases other than those for which they were originally created? Rather than have a scientist sort through the results, we can use text analysis to get a loose idea of this picture in relatively short order.

*Note: The order below is more of a logical workflow than what actually needs to happen for your code. If you want to turn anything into standalone functions, you may want to shuffle things around. If you want to translate your code to a workable script, it will likely need to be split into various functions.*

--------------
## Imports

What packages / modules do you need? Those mentioned below are just those that I use in the current version of my workflow. Feel free to add/delete as you see fit.

Most of these are probably in the standard Anaconda distribution, but you may need to add/update certain packages.

In [9]:
# os - https://docs.python.org/3/library/os.html
# for interacting with your computer's file system
import os

#-------------------------------
# pandas - https://pandas.pydata.org/docs/user_guide/index.html
# for structuring and shaping your data
import pandas as pd
# command for Jupyter to display more than the default 50 pandas rows at once. Adjust number to preference
pd.set_option('display.max_rows', 500)

#-------------------------------
# re - https://docs.python.org/3/library/re.html
# for building custom regular expressions for use in text pre-processing
import re

#-------------------------------
# string - https://docs.python.org/3/library/string.html
# for removing punctuation and other misc. string operations during pre-processing
import string

#-------------------------------
# Natural Language Toolkit - https://www.nltk.org/
# Expansive library for working with natural language / free text data

# modules
# Stopwords corpus - https://www.nltk.org/nltk_data/
# Provides access to lists of common stopwords
from nltk.corpus import stopwords as english_stopwords
# Word tokenizer - https://www.nltk.org/api/nltk.tokenize.html
# Breaks a string into a list of substring tokens
from nltk.tokenize import word_tokenize
# lematizer - https://www.nltk.org/_modules/nltk/stem/wordnet.html
# Reduces tokens to their 'lemma' (base grammatical forms) as opposed to their stems (base form without affixes, igorant of grammatical context)
from nltk.stem import WordNetLemmatizer
# Part-of-speech tagger - https://www.nltk.org/api/nltk.tag.html
# Processes a sequence of words, and attaches a part of speech tag to each word (Used as input for lemmatizer)
from nltk import pos_tag
# Wordnet - https://www.nltk.org/howto/wordnet.html
# Used to convert pos_tag output into a form that can be used as input for the lemmatizing operation
from nltk.corpus import wordnet
# Porter Stemmer - https://www.nltk.org/_modules/nltk/stem/porter.html
# Applies the Porter-Stemming operation to tokens. Reduces words to their base, affix-less form, but loses some morphological associations
from nltk.stem import PorterStemmer

#-------------------------------
# SciKit Learn - https://scikit-learn.org/stable/
# Expansive machine learning library

# modules (algorithms) :
# Latent Dirichlet Allocation (LDA) - Not really a clustering operation. Maps topics over sets of terms, documents assigned to topics based on term-alignment with topic terms
from sklearn.decomposition import LatentDirichletAllocation
# other clustering algorithms here: https://scikit-learn.org/stable/modules/clustering.html#clustering

# modules (utilities)
# CountVectorizer - https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.CountVectorizer.html
# Transforms string documents into a term-frequency (TF) matrix. (Only type of input accepted by LDA)
from sklearn.feature_extraction.text import CountVectorizer
# TFIDF Vectorizer - https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.TfidfVectorizer.html
# Transforms string documents into a term-frequency inverse-document-frequency (TFIDF) matrix. Many algorithms accept both TF and TFIDF, but TFIDF is more nuanced
from sklearn.feature_extraction.text import TfidfVectorizer

# sparse matrix library
# for stopword generation
from scipy.sparse import coo_array, csr_array, csc_array, csr_matrix, coo_matrix, csc_matrix
import scipy.sparse as sp

#-------------------------------
# time - https://docs.python.org/3/library/time.html
# for timing operations
from time import time

#-------------------------------
# numpy - https://numpy.org/
# for working with array formats used as input/output by certain SciKit Learn operations and accessory functions
import numpy as np

#-------------------------------
# PyLDAvis (Latent Dirchlet Allocation visualization for Python) - https://pyldavis.readthedocs.io/en/latest/readme.html
# Excellent library for visualizing the output of SciKit Learn LDA models
import pyLDAvis.sklearn

# --------------------
# Warnings - https://docs.python.org/3/library/warnings.html
# Not essential. Using this function prevents deprecation warnings, etc. from cluttering your notebook.
import warnings
warnings.filterwarnings('ignore')

import scipy

-----------------
## Load data

Import your source data. What columns are important?

In [10]:
citations = pd.read_csv("all_soybean_citations.csv")

In [11]:
citations = citations[~citations.abstract.isna() & ~citations.title.isna()]

In [12]:
citations.columns

Index(['id', 'title', 'abstract', 'altmetric', 'authors', 'category_for',
       'category_rcdc', 'category_uoa', 'concepts_scores', 'date', 'doi',
       'funders', 'open_access', 'pages', 'reference_ids',
       'research_org_cities', 'research_org_countries',
       'research_org_country_names', 'research_orgs', 'researchers',
       'times_cited', 'type', 'volume', 'year', 'journal.id', 'journal.title',
       'research_org_state_codes', 'research_org_state_names', 'pmcid',
       'category_bra', 'issue', 'pmid', 'category_hrcs_hc', 'category_hra',
       'category_hrcs_rac', 'category_sdg', 'field_citation_ratio',
       'relative_citation_ratio', 'category_icrp_cso', 'category_icrp_ct'],
      dtype='object')

In [13]:
data = citations.copy()

Many of the above columns will be extraneous for your purposes. All of the 'category' fields are automatically derived by the data provider I use (Dimensions), so I wouldn't try to incorporate these into my models unless you've worked with the titles + abstracts and are curious as to how these other fields might affect your results. In general, I try to avoid running machine learning operations on text results derived from other automated processes, as this can add layers of abstraction that may be difficult to explain or replicate.

*Note that some steps below reference the Pandas DataFrame variable called __'data'__. In my workflow, I return a copy of my original 'citations' DataFrame after my preprocessing steps, which includes a column called 'txt' to store my processed text for each document.*

----------
## Pre-process text

What does your text need to look like to ensure that your algorithm will provide meaningful results?

__Perhaps a good first step is to ignore pre-processing altogether. Simply send your raw text to the vectorizer and see what your algorithm spits out. How does it look? What would you change?__

Some possible routes to explore:
* Noise / formatting cleanup (html tags, copyright notices, URLs, standalone punctuation/numbers, etc.)?
* Tokenization? Case-standardization (lower-casing)?
* Stopword removal?
  * General English?
  * General scientific literature? (let me know if you want me to send you my go-to list, or come up with one on your own)
  * Soybean research: domain-specific?
* Word root reduction (stemming / Lemmatization)?
* Disambuguating organism names / abbreviations? What about chemical names / abbreviations?
* Eliminating domain-specific affixes not caught by your stemmer/lemmatizer (i.e. bacterial > bacteria)?
* Dealing with word-internal punctuation?
* Weighting certain terms over others (i.e. title terms vs. abstract terms)?
* Other considerations??

*Note that the data format I use for input into the SciKit Learn vectorizers is generally a Pandas series of string values, so this is the output format I'd aim for with any pre-processing setup.*

In [52]:
txt = data.abstract

In [53]:
# Filter out html
txt = txt.str.replace(r'<[^<>]*>', '', regex=True)

In [54]:
# Filter out stopwords
txt = txt.str.replace(r'Background(?!\ )', '')
txt = txt.str.replace(r'Method(?!\ )', '')
txt = txt.str.replace(r'Title(?!\ )', '')

In [55]:
# Manually filter multigrams
txt = txt.str.replace(r'[Qq]uantitative\W[Tt]rait\W[Ll]oc[i|us]+|[Qq][Tt][Ll][sS]|[Qq][Tt][Ll]', 'QTL')
txt = txt.str.replace(r'[Ll]inkage\W*[mM]ap\w*', 'linkage_map')

In [56]:
txt.sample().values

array(['Pigeonpea (Cajanus cajan L. Huth) is an agronomically important legume cultivated worldwide. In this study, we extensively analyzed gene-body methylation (GbM) patterns in pigeonpea. We found a bimodal distribution of CG and CHG methylation patterns. GbM features- slow evolution rate and increased length remained conserved. Genes with moderate CG body methylation showed highest expression where as highly-methylated genes showed lowest expression. Transposable element (TE)-related genes were methylated in multiple contexts and hence classified as C-methylated genes. A low expression among C-methylated genes was associated with transposons insertion in gene-body and upstream regulatory regions. The CG methylation patterns were found to be conserved in orthologs compared with non-CG methylation. By comparing methylation patterns between differentially methylated regions (DMRs) of the three genotypes, we found that variably methylated marks are less likely to target evolutionary co

In [57]:
# Replace all species names abbreviations with species names, 
# Don't try to figure out this code

def ret_match(m):
    return m.group(1) + '_' + m.group(2) if m else np.NaN

def replace_abvs(abvs, abstract):
    for k, v in abvs:
        abstract = re.sub(k, v, abstract)
    return abstract

abvs = txt.str.extractall('(([A-Z])\.\ ([a-z]\w+))').drop_duplicates()
abvs[3] = abvs.apply(lambda x: f'({x[1]}\w+) ({x[2]})', axis=1)
abvs[4] = abvs.apply(lambda x: f'({x[1]}[\w|.]+ {x[2]})', axis=1)
tmp = abvs.groupby(level=0).apply(lambda x: list(x[3])).to_frame().explode(0)
tmp[1] = txt
tmp = tmp.apply(lambda x: ret_match(re.search(x[0], x[1])), axis=1)
tmp = pd.concat([abvs[4].groupby(level=0).apply(list).explode(), tmp], axis=1).dropna()
tmp[1] = list(zip(tmp[4], tmp[0]))
tmp = pd.concat([tmp[1].groupby(level=0).apply(list), txt], join='inner', axis=1)
flt = tmp.apply(lambda x: replace_abvs(x[1], x['abstract']), axis=1)
txt.loc[flt.index] = flt

In [58]:
# lowercase
txt = txt.str.lower()

In [59]:
# filter punctuation
txt = txt.str.replace('-', ' ')
txt = txt.str.replace('\W+', ' ')

In [60]:
# split individual terms
words = txt.str.split(' ').explode()

In [61]:
words = words[words.str.contains('^[a-z]')]

In [62]:
stemmer = PorterStemmer()
words = words.apply(stemmer.stem)

In [63]:
txt = words.groupby(level=0).apply(' '.join)

In [64]:
txt.sample().values

array(['soybean glycin max l merr is one of the most import oil and protein crop ever increas soybean consumpt necessit the improv of varieti for more effici product howev both correl among differ trait and genet interact among gene that affect a singl trait pose a challeng to soybean breed resultsto understand the genet network underli phenotyp correl we collect soybean access worldwid and phenotyp them for two year at three locat for agronom trait genom wide associ studi identifi signific genet loci among which genet interact with other loci we determin that oil synthesi relat gene are respons for fatti acid accumul in soybean and function in line with an addit model network analys demonstr that trait could be link through the linkag disequilibrium of associ loci and these link reflect phenotyp correl we reveal that loci includ the known dt1 e2 e1 ln dt2 fan and fap loci as well as undefin associ loci have pleiotrop effect on differ trait conclusionsthi studi provid insight into the 

--------------
## Vectorize your text data

Transform your processed text into a machine-readable, numeric array. What parameters make the most sense for your data?

The example vectorizer below is a [Count Vectorizer](https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.CountVectorizer.html). Another vectorizer commonly used with text data is a [TFIDF vectorizer](https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.TfidfVectorizer.html).  See [this site](https://scikit-learn.org/stable/modules/feature_extraction.html) for more documentation on feature extraction with SciKit Learn.

Parameters used below:
* __Maximum document frequency__. This parameter removes terms from your feature matrix that occur in more than X% of your documents. A value of .98 eliminates words that occur in more than 98% of all corpus documents. This removes noisy words that don't differentiate documents.
* __Minimum document frequency__. This parameter removes words from your feature matrix that occur in fewer than X documents in your corpus. A value of 3 eliminates words that occur in 2 or fewer documents. This removes many unique words that don't help associate documents and reduces the dimensionality (# columns) of your matrix to improve compute times.
* __NGram range__. A 'gram' is a term (column) in your frequency matrix, and an 'NGram' is a term made up of N tokens (individual words). Unigrams (N=1) consist of single tokens. Higher N values increase the number of tokens in direct proximity to one another that are strung together as single terms. Consider the phrase 'laser beam frequencies'. This phrase is made up of 3 unigrams ('laser', 'beam', 'frequencies'), 2 bigrams ('laser beam', 'beam frequencies'), and 1 trigram ('laser beam frequencies'). Your NGram range is the set of all N values you'd like to include in your feature matrix. A range of (1,1) only includes unigrams, while a range of (1-5) would generate features for unigrams, bigrams, trigrams, 4-grams, and 5-grams. Multigrams (higher than 1) contain important word associations that can be informative for your model in your specific domain context, but they also contain many useless phrases that serve only as noisy data. It is also important to consider that including multigrams drastically increases the dimensionality of your feature matrix and can slow down your model-generation process. While unigrams are crude, I find that they often lead to the best results.
* __Stopwords__. Instructs the vectorizer to eliminate features matching common stop_words (the 'english' set is used below). This is helpful to catch anything you missed in pre-processing, but not required.

*NOTE: The values used below are not final. You should adjust them as you see fit. Also, there are many other parameters you can use to adjust your vectorizer. Feel free to experiment with other parameters.*


In [83]:
%%time

# initialize your vectorizer using desired paramters. Think of this step as 'flipping the on switch' and turning some knobs
tf_vectorizer = CountVectorizer(max_df=1., # Maximum document frequency
                                min_df=3, # Minimum document frequency
                                max_features=None, # Limits the size of your array by only including the top N features by term frequency. Not used here (handy for huge datasets)
                                ngram_range=(1, 1), # NGram range
                                stop_words='english')

# Create a feature representation using the vectorizer. 
# The output is a sparse matrix that can be used as input for many SciKit Learn clustering algorithms.
# Here, 'data' represents a copy of my 'citations' dataframe. The 
# series 'txt' has been added by my preprocessing steps as a place for
# storing the processed versions of my title / abstract text
tf_vectorizer.fit(txt)
features = tf_vectorizer.transform(txt)
features

CPU times: user 739 ms, sys: 13.3 ms, total: 753 ms
Wall time: 754 ms


<5293x6997 sparse matrix of type '<class 'numpy.int64'>'
	with 449838 stored elements in Compressed Sparse Row format>

-----
## Automatic Stopword Detection

input a tf_matrix and txt and output stopwords

In [84]:
def calc_term_entropy(tf_matrix):
    H = np.zeros(tf_matrix.shape[1])
    
    tf_matrix = coo_array(tf_matrix) # row col access
    tf_wc = tf_matrix.sum(axis=0)    # TF(w, C)
    
    for d, w, tf in zip(tf_matrix.row, tf_matrix.col, tf_matrix.data):            
        p_dw = tf / tf_wc[w]
        H[w] -= p_dw * np.log2(p_dw)
    
    return H

def calc_null_entropy(tf_vectorizer, words, doc_idx, random_rounds=5):
    if words.shape != doc_idx.shape:
        return None
    
    null_entropy = np.zeros(len(tf_vectorizer.get_feature_names()))
    
    random_rounds = 5
    for i in range(0, random_rounds):
        np.random.shuffle(words)
        null_txt = pd.Series(words, doc_idx).groupby(level=0).apply(' '.join).values
        null_entropy += calc_term_entropy(tf_vectorizer.transform(null_txt))

    return null_entropy / random_rounds
    
def normalize(arr):
    mx = arr.max()
    mn = arr.min()
    return (arr - mn) / (mx - mn)

In [85]:
entropy = calc_term_entropy(features)
vocabulary = tf_vectorizer.get_feature_names()

In [91]:
words = txt.str.split(' ').explode()
null = calc_null_entropy(tf_vectorizer, words.values, words.index.values)
inform = entropy * (1 - normalize(null - entropy))
tf = features.sum(axis=0).A[0]
raw_downshift = (scipy.stats.rankdata(inform) - scipy.stats.rankdata(tf))
downshift = raw_downshift / len(inform)
semantic_entropy = inform + downshift * np.log(len(inform))

In [92]:
stopwords = pd.DataFrame(index=vocabulary)
stopwords['tf'] = tf
stopwords['entropy'] = entropy
stopwords['information'] = inform
stopwords['sm'] = null - entropy

In [93]:
stopwords['downshift'] = downshift

In [94]:
stopwords['experimental'] = semantic_entropy

In [121]:
n = 100
p = 0
stoptable = pd.DataFrame()
ranktable = pd.DataFrame(index=stopwords.index)

metrics = ['tf', 'entropy', 'information', 'experimental']
for metric_name in metrics:
    metric = stopwords[metric_name].sort_values(ascending=False)
    stoptable[metric_name] = metric.index
    stoptable[metric_name + '_value'] = metric.values
    ranktable[metric_name] = stopwords[metric_name].rank(ascending=False, method='min').values.astype('int')

In [122]:
tf_shift = 1 - normalize(stopwords['tf'].rank(ascending=False) - stopwords['information'].rank(ascending=False))
tf_shift *= tf
tf_shift = tf_shift.sort_values(ascending=False)
stoptable['shift'] = tf_shift.index
stoptable['shift_value'] = tf_shift.values

In [123]:
tf_shift

gene              4443.446185
soybean           3282.331841
plant             2432.237767
genom             2003.177206
express           1712.562209
                     ...     
endosom              0.010446
aggress              0.010446
mesorhizobium        0.010446
proteomexchang       0.000000
mainstream           0.000000
Length: 6997, dtype: float64

In [126]:
stoptable[['tf', 'entropy', 'information', 'experimental']]

,tf,entropy,information,experimental
0,gene,gene,thi,thi
1,soybean,thi,studi,studi
2,plant,plant,result,result
3,genom,studi,provid,provid
4,express,soybean,import,import
...,...,...,...,...
6992,chenopodium,stf1,ppo,np
6993,chen,abi4,qph,al
6994,chemodivers,cpk,ahl,gmfad2
6995,hd2,gmbzip1,aerenchyma,melatonin


In [130]:
n = 2000
stoptable[['tf', 'entropy', 'information', 'experimental']][:n].to_html('pages/stoptable.html', index=True)
ranktable.sort_values('tf')[:n].to_html('pages/ranktable.html')

In [108]:
stopword_mask = semantic_entropy > np.percentile(semantic_entropy, 80)

In [109]:
stopword_mask.sum()

1377

In [110]:
stopwords_set = np.ma.compressed(np.ma.masked_array(np.array(vocabulary), mask=~stopword_mask))

In [111]:
    print(stopwords_set)

['aberr' 'abil' 'abiot' 'abl' 'abov' 'abscis' 'absenc' 'absent' 'abstract'
 'abstractsoybean' 'abstractth' 'abund' 'acceler' 'accept' 'access'
 'accompani' 'accord' 'accordingli' 'account' 'accumul' 'accur' 'acet'
 'achiev' 'acid' 'acquir' 'act' 'action' 'activ' 'actual' 'ad' 'adapt'
 'add' 'addit' 'address' 'adjac' 'adjust' 'adopt' 'advanc' 'advantag'
 'advent' 'advers' 'affect' 'agent' 'agreement' 'agricultur' 'agronom'
 'aid' 'aim' 'allevi' 'allow' 'alon' 'alreadi' 'alter' 'altern' 'altogeth'
 'alway' 'amen' 'america' 'amino' 'ammonia' 'amplifi' 'analys' 'analysi'
 'analyz' 'anatom' 'ancestor' 'ancestr' 'ancient' 'ani' 'anim' 'annot'
 'anoth' 'answer' 'anticip' 'apart' 'aphi' 'appar' 'appear' 'appli'
 'applic' 'approach' 'approxim' 'arabidopsi' 'arabl' 'area' 'aris'
 'arrang' 'art' 'articl' 'artifici' 'asian' 'aspect' 'assay' 'assess'
 'assign' 'assist' 'associ' 'ataf1' 'attempt' 'attent' 'attract'
 'attribut' 'automat' 'avail' 'avenu' 'averag' 'avoid' 'background'
 'bacteria' 'bala

In [103]:
(~stopword_mask).sum()

6542

In [104]:
(~stopword_mask).astype(int).sum()

6542

In [105]:
features = features @ sp.diags((~stopword_mask).astype(int))

In [106]:
features

<5293x6997 sparse matrix of type '<class 'numpy.float64'>'
	with 196488 stored elements in Compressed Sparse Row format>

In [107]:
v = entropy * (1 - normalize(raw_downshift))
pd.Series(v, vocabulary).sort_values(ascending=False)[:100]

mirna         6.925958
flood         6.084591
scn           5.263805
nodul         5.182777
al            5.158064
aphid         4.718644
smv           4.606956
iron          4.310724
fe            4.288097
rhg1          4.182601
salt          4.161163
cd            4.157764
drought       4.142217
isoflavon     4.087264
qtl           4.053118
methyl        4.027462
pi            4.004814
te            3.921734
wrki          3.867014
resist        3.803878
et            3.797016
nb            3.783909
chickpea      3.682616
fad2          3.666405
lncrna        3.656203
bean          3.649178
mg            3.638210
oil           3.634415
flower        3.601925
mapk          3.599269
gm            3.587322
br            3.586169
rlk           3.578217
peptid        3.575505
snp           3.534400
cle           3.530065
nil           3.496153
saponin       3.489578
er            3.448505
nf            3.446197
pa            3.430143
seed          3.426410
deg           3.410346
ltr        

In [ ]:
v.argsort()
np.array(vocabulary)[v.argsort()[::-1]].tolist()

In [ ]:
stopword_mask

In [ ]:
vocabulary

In [ ]:
np.ma.compressed(np.ma.masked_array(np.array(vocabulary), mask=stopword_mask))

--------------
## Run your algorithm

It has all led up to this moment! Here's where you get to see what the algorithms make of your processed text.

Here are some algorithms you can try (not exhaustive):
* KMeans - https://scikit-learn.org/stable/modules/generated/sklearn.cluster.KMeans.html
* Latent Dirichlet Allocation (LDA) - https://scikit-learn.org/stable/modules/generated/sklearn.decomposition.LatentDirichletAllocation.html
* DBSCAN - https://scikit-learn.org/stable/modules/generated/sklearn.cluster.DBSCAN.html#sklearn.cluster.DBSCAN
* Nonnegative Matrix Factorization (NMF) - https://scikit-learn.org/stable/modules/generated/sklearn.decomposition.NMF.html. This one is actually not a clustering algorithm either, but is designed for dimensionality reduction. However it can be leveraged for clustering in the same way that LDA can. Let me know if you want to try this one and I can go dig up some sample code for making it work

|||||||||||||||||||||
### *Notes on LDA example below*

The algorithm used in the example below is called Latent Dirchlet Allocation (LDA). If you're interested, the original publication describing the algorithm is [here](https://dl.acm.org/doi/pdf/10.5555/944919.944937).

LDA isn't *actually* a clustering algorithm. It maps a series of topics over a corpus of words/documents. To leverage this tool as a clustering algorithm, you need to assign documents to topics (clusters) by looking at the "probability that a certain document belongs to a certain topic; this is based on how many words (except the current word) from this document belong to the topic of the current word." - See this [Towards Data Science page](https://towardsdatascience.com/latent-dirichlet-allocation-for-topic-modelling-explained-algorithm-and-python-scikit-learn-c65a82e7304d) for a good breakdown of what's going on when you implement LDA in SciKit Learn.

For each document, you'll end up with a series of float values denoting probabilities for association with each topic. Using some simple code, we can 'tag' our citations in our original dataframe with the topics they've been assigned to.

Parameters used below:

* Number of Components (usually called K) - How many topics/clusters do you want in your model? The ideal setting for this parameter depends on the actual trends in your data, as well as how granular you want to be in your investigation. (*There is an automated method called the 'Elbow Method' for automatically determining an optimal K value for a given dataset, but your mileage may vary as clustering is not an exact science - [short description can be found here](https://towardsdatascience.com/k-means-clustering-algorithm-applications-evaluation-methods-and-drawbacks-aa03e644b48a)*)
* Max iterations - How many topic reassignment iterations do you want your algorithm to perform? Higher values take longer, but can lead to more coherent topics.
* Learning method - I'm not sure about the specifics here, but 'online' (as opposed to 'batch') for this parameter lowers the compute time for larger datasets. You can read more about this with SciKit Learn's documentation.
* Random state - LDA uses random assignments in initial passes, and passing a consistent random state to the algorithm allows you to reproduce your results over multiple function calls. More on this in the documentation if you're interestred.
* Batch size - Again I'm not exactly sure on the specifics here because this is only used for the 'online' learning method. I think the basic idea is that you can control how many documents are being included in smaller samples pulled to make the iterations go faster.
* Number of Jobs - This tells the algorithm how many of your computer's workers (cores) it can occupy while running. A good rule of thumb is n - 1, where n is the number of logical cores on your computer (set to 7 below because my personal laptop has 8 logical cores). Should you move your operation to an HPC environment, this parameter becomes much more important.

*NOTE: The values used below are not final. You should adjust them as you see fit. Also, there are many other parameters you can use to adjust your algorithm, and each algorithm has different parameters. Feel free to experiment with other parameters.*


In [ ]:
%%time

# initialize your algorithm using desired paramters. Think of this step as 'flipping the on switch' and turning some knobs
lda = LatentDirichletAllocation(n_components=15,           # Number of topics
                                  max_iter=50,               # Max learning iterations. LDA tries to solidify randomly assigned topics over many iterations, and this prevents 'run-away' operations and long compute times
                                  learning_method='online',  # Learning method 
                                  random_state=100,          # Random state
                                  batch_size=128,            # n docs in each learning iter
                                  n_jobs = 7,               # Use n_jobs workers
                                 )



# This is the operation that actually fits your model to your data
# It returns a matrix with X columns and Y rows, where X is your # of topics/clusters ('n_components')
# and Y is your total number of documents. Each cell contains a float probability value for a given
# document's (row) relatedness to a given topic (column)
%time lda_output = lda.fit_transform(features)

----------
## Sanity checks / model evaluation

This is where the real finesse comes in. The goal here is to look at your resulting model and to determine how it can be improved (or if it has reached a 'good enough' state - no model will ever be perfect). This step can be similar for every corpus you work with, but the conclusions you draw and steps you take to address those conclusions will differ each time you work with a new corpus.

Some questions to guide you in this step:
* Do these topics actually make sense? Would you be able to confidently describe these results to another person?
* Are there false equivalencies or distinctions being made? Am I getting more granular with my topics than the data actually allows? Am I grouping things that shouldn't be grouped? Why?
* Take a look at the actual topic assignments for a few documents. What publications are actually being grouped together? Can the top/prominent words in a given cluster at least generally relate to the topic of a selected publication?

There are many ways to go about this. Some general approaches I use (or hope to use in the future) for every dataset:

* What are the top words in each of your clusters? Are there similarities? Is noisy/artifactual data showing up?
* How many documents out of your corpus are assigned to each cluster? Are they all being lumped into 2-3 big topics while the smaller topics only get tiny numbers of documents? Are they more evenly distributed?
* Do your resulting topics seem too broad? Too specific? You may need to adjust your n_components parameter.
* Try visualizing your clustering results. The methods for doing so differ based on the output of each algorithm, so you may need to get creative with your code (LDA example below)
* Are there any quantitative measures you can use to evaluate your clustering results? One example is the ['Silhouette Method' for KMeans models](https://scikit-learn.org/stable/auto_examples/cluster/plot_kmeans_silhouette_analysis.html), which attempts to determine distance separation between clusters. This method can tell you if a larger or smaller K value might be useful.

||||||||||

### *Surface Level - documents in topics / terms in topics*

Below is some sample code for A) assigning your documents to clusters, B) looking at the most prominent words in a given topic, and C) seeing how many documents are in a given cluster. I usually do this as a first sanity check for LDA output before sending my results to pyLDAvis.

The first step is to assign your original citations to clusters/topics based on your model's output:

In [ ]:
# Create a dataframe where the index (Y) is the docs, and the columns (X) are the topics
# Assign "clusters" based on highest topic association probability for each doc

# First step: create a blank dataframe with K columns and N rows, where K is the number of topics (n_components)
# and N is your total number of documents
topics = [str(i) for i in range(lda.n_components)]
docs = [str(i) for i in range(data.shape[0])]

# Populate this blank dataframe using your LDA output. What you're doing is here is translating the
# raw lda output (a numpy.ndarray of numpy.ndarrays) into the rows/columns you established above
document_topic_df = pd.DataFrame(np.round(lda_output, 2), columns=topics, index=docs)

Now we'll use Numpy's argmax function to assign documents to topics. What's happening here is that the function is examining all of the probability values in a given row, and picking the column (topic) with the highest probability for a given row (document). Unfortunately, when there is a tie, [np.argmax](https://numpy.org/doc/stable/reference/generated/numpy.argmax.html) picks the 'first' highest value it finds. If you want to instead randomize topic assignments in the case of ties, you might need to improvise (a good Stack Overflow thread on that can be found [here](https://stackoverflow.com/questions/42071597/numpy-argmax-random-tie-breaking)).

The code below essentially returns a Numpy ndarray of highest-probability topics for all of your documents.

In [ ]:
labels = np.argmax(document_topic_df.values, axis=1)

Thankfully, your topic assignments ndarray above is in the same order as your original dataframe, so you can simply add those assignments as a new column to your 'citations' dataframe:

In [ ]:
# By 'labels' - we're referring to topics
citations['label'] = labels

Now we're going to see how many documents are in each topic. This is a nice baseline look at your document distribution across your model. One nasty habit I see with a lot of LDA models I build is that the lion's share of documents will be placed in a select few topics, while the remaining topics will be very small and meaningless. This is generally a sign that you need to make some adjustments.

The code below uses the Pandas value_counts() function on the 'labels' column of your citations data frame to make a new dataframe, where one column is your topic #, and the other is your document frequencies.

In [ ]:
ranks = pd.DataFrame(citations['label'].value_counts()).reset_index()
ranks.index += 1
ranks.columns = ['topic', 'docs']
all_ranks = ranks.sort_values(by="topic").reset_index(drop=True)
all_ranks

Now that you can see where your documents have landed, let's take a look at what words are most prominent in each topic. If you see nonsense or noise showing up as 'prominent' in a given topic, it's probably time for some further tuning.

The code below makes use of a function - 'show_topics' - that I found online (can't remember the link). I don't remember exactly what's going on, but suffice it to say that the function takes your model, vectorizer, and the # of words you want to see as input, and returns a list of ndarrays containing top terms, one for each topic, as output. In the example below, I chose to return the top 15 terms for each topic.

In [ ]:
def show_topics(vectorizer, lda_model, n_words):
    keywords = np.array(vectorizer.get_feature_names())
    topic_keywords = []
    for topic_weights in lda_model.components_:
        top_keyword_locs = (-topic_weights).argsort()[:n_words]
        topic_keywords.append(keywords.take(top_keyword_locs))
    return topic_keywords

topic_keywords = show_topics(tf_vectorizer, lda, 15)

Now we can translate that list of ndarrays into a dataframe for easy viewing. The code below does so and also assigns some cleaner column/row names.

In [ ]:
df_topic_keywords = pd.DataFrame(topic_keywords)
df_topic_keywords.columns = ['Word '+str(i) for i in range(df_topic_keywords.shape[1])]
df_topic_keywords.index = ['Topic '+str(i) for i in range(df_topic_keywords.shape[0])]

Sometimes you'll find situations in which certain topics don't get any documents assigned to them. We'll want to see those too, so the code below adds them to your topic dataframe. It also adds the # of documents assigned to each topic, as determined above.

In [ ]:
all_ranks = ranks.sort_values(by="topic").reset_index(drop=True)
clusters = 15

for i in range(0, clusters):
    if i not in all_ranks['topic'].values:
        all_ranks = pd.concat([all_ranks, pd.DataFrame([{'topic':i, 'docs':0}])])

df_topic_keywords['docs'] = list(all_ranks['docs'])
df_topic_keywords

||||||||||

### *LDA Visualization Example*

I use PyLDAvis extensively every time I use LDA. It's a really handy way to get an overhead view of your terms/clusters and the relationships between them. You can read more about how it works [here](https://pyldavis.readthedocs.io/en/latest/readme.html).

[This article on PyLDAvis](https://towardsdatascience.com/topic-model-visualization-using-pyldavis-fecd7c18fbf6) is was paywalled for me, but I think you can access it with a free account. You may get some useful tips from it. [This notebook](https://nbviewer.org/github/bmabey/pyLDAvis/blob/master/notebooks/pyLDAvis_overview.ipynb#topic=0&lambda=1&term=), linked on the package documentation site, has a lot of handy code examples and descriptions.

Arguments used below:

* 'lda' is the variable for your initialized algorithm (not the results)
* 'features' is the variable for the sparse matrix returned by your vectorizer
* 'tf_vectorizer' if the variable for your initialized vectorizer
* 'mds' designates your multidimensional scaling method. Because the resulting visualization is displayed on a 2D plane, you have to reduce your multi-dimensional feature to something you can actually look at. There are several options here, and I don't know the specifics on how they differ, but 'tsne' has always worked well for me.

In [ ]:
# Initialize your visualization.
panel = pyLDAvis.sklearn.prepare(lda, features, tf_vectorizer, mds='tsne')

In [ ]:
# Create an inline display that shows up directly in your Notebook
pyLDAvis.display(panel)



# Note that if you use this command with JupyterLab, all of your Jupyter UI buttons will disappear.
# The alternative is pyLDAvis.show(), which opens an HTML version of the visual in a new tab.
# Unfortunately the most recent version of pyLDAvis has an issue with its
# html generation, so it's probably safer to use .display() with Jupyter Notebooks instead.

In [ ]:

pyLDAvis.save_html(panel, 'pages/lemmatized.html')

In [ ]:
for i, abstract in enumerate(citations.abstract[txt.str.contains('biofuel')].sample()):
    with open(f'sample_texts/abstract_{i}.txt', 'w') as f:
        print(abstract)
        f.write(abstract)

------
## Future Directions

If you get to where you are happy with your model, there are plenty of other directions in which you could take your results.

We can talk specifics on any of these (or anything you come up with on your own) when we get there, but here are some starter thoughts:

* Dynamic topic visualization. Could we build something that would allow folks to interactively explore the model you've built? What are some subtopics in each major cluster? Where are the authors of papers in that cluster coming from? What are some notable examples of papers/outcomes that are grouped in a given model?
* Connecting these models with community analysis for authors on the publications. Can we break down the authors on these papers into subcommunities? How do those subcommunities correspond or overlap with the topics/clusters from your model? Could we connect a topic visualization with a network visualization?
* Prediction. Could we explore some applications involving supervised learning on your processed data? What classes within these publications might be worth predicting? Publishing journals? Citation thresholds? High vs. low media attention?
* Change over time. How does your model look if we add in a temporal element? Are certain topics gaining/losing prominence over time?
* Proposal analysis. What does it look like if we throw the original Soybean proposal text into the mix? Can we get some idea of which papers are closely aligned with the original proposal and which papers are not?
* More publications. How widely applicable is your model? If I provided a larger, more general set of publications dealing heavily with soybean research, would you need to adjust your parameter tuning or preprocessing steps? If not, how close are the topics/clusters in the larger topic to those you generated just from those papers citing JGI's soybean genome?




